In [1]:
from jax.config import config
config.update("jax_enable_x64", True)

import jax.numpy as jnp
from jax import jit, lax, vmap, jacobian
import jax.scipy.linalg as jsl

import jax

import numpy as np
from read_traj_v2 import *

import scipy.optimize

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
# !pip3 install torch torchvision torchaudio

In [3]:
import torch
import torch.nn as nn

In [4]:
mol = 'heh+'
basis = 'sto-3g'
init = 'hf'
td = 'rt-tdexx'

dt = 0.08268/10

inpath = './correctdata'
inpath2 = './mydata/'
outpath = './'
saved_model_dir = './models/'

trnfrq = jnp.array(np.load('./tdcis_data/tdcis_trnfreq_dt0.008268.npz'))
trnamp = jnp.array(np.load('./tdcis_data/tdcis_trnamp_dt0.008268.npz'))
trntme = jnp.array(np.load('./tdcis_data/tdcis_trntme_dt0.008268.npz'))
# load important static matrices
a = traj_data(inpath, mol, basis, init, td, '001',
                init_cond_file=f'ke+en+overlap+ee_twoe+dip_casscf22_{mol}_{basis}.npz',
                )
a.assign_init_data()
drc = a.drc
xmat, didat, kinmat, enmat = a.xmat, a.didat, a.kinmat, a.enmat
X = xmat


	reading from "./correctdata/ke+en+overlap+ee_twoe+dip_casscf22_heh+_sto-3g.npz"

can only concatenate str (not "NoneType") to str
True is not in list


In [5]:
trnden = np.array(np.load('./tdcis_data/tdcis_dt0.0082680.npz'))
print(trnden.shape)

(301, 10000, 2, 2)


In [6]:
trntimecoeffs = np.array(np.load('./tdcis_data/time_coeffs_tdcis_dt0.008268.npz'))
print(trntimecoeffs.shape)

(300, 10000, 2)


In [7]:
torch.set_default_dtype(torch.float64)
input_size = 4
output_size = 4
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        # layer 1
        self.fc1 = nn.Linear(input_size, 100) 
        self.batch1 = nn.BatchNorm1d(100)
        self.drop1 = nn.Dropout(0.25)

        # layer 2
        self.fc2 = nn.Linear(100, 80)
        self.batch2 = nn.BatchNorm1d(80)
        self.drop2 = nn.Dropout(0.25)

        # layer 3
        self.fc3 = nn.Linear(80, 60)
        self.batch3 = nn.BatchNorm1d(60)
        self.drop3 = nn.Dropout(0.25)

        # layer 4
        self.fc4 = nn.Linear(60, 40)
        self.batch4 = nn.BatchNorm1d(40)
        self.drop4 = nn.Dropout(0.25)

        self.fc5 = nn.Linear(40, output_size)
        
        self.selu = nn.SELU() 
        self.sm = nn.Softmax(dim=1)
    def forward(self, x):                            

        # layer 1
        out = self.fc1(x)
        out = self.batch1(out)
        out = self.selu(out)
        out = self.drop1(out)

        # layer 2
        out = self.fc2(out)
        out = self.batch2(out)
        out = self.selu(out)
        out = self.drop2(out)

        # layer 3
        out = self.fc3(out)
        out = self.batch3(out)
        out = self.selu(out)
        out = self.drop3(out)

        # layer 4
        out = self.fc4(out)
        out = self.batch4(out)
        out = self.selu(out)
        out = self.drop4(out)

        out = self.fc5(out)
        out = self.sm(out)
        return out


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('using gpu')

using gpu


In [9]:
#TODO split into real and imaginary parts
X = trnden[140].reshape((10000,4))
y = trntimecoeffs[140]
X.shape, y.shape

((10000, 4), (10000, 2))

In [10]:
X

array([[0.10477089+0.00000000e+00j, 0.30625791+0.00000000e+00j,
        0.30625791+0.00000000e+00j, 0.89522938+0.00000000e+00j],
       [0.10477089+0.00000000e+00j, 0.30625791+0.00000000e+00j,
        0.30625791+0.00000000e+00j, 0.89522938+0.00000000e+00j],
       [0.10477089+6.78269230e-24j, 0.30625792-1.90978295e-06j,
        0.30625792+1.90978295e-06j, 0.89522937-1.05200500e-23j],
       ...,
       [0.11820466-2.04482806e-18j, 0.31036451-8.85466911e-02j,
        0.31036451+8.85466911e-02j, 0.88179556-2.15445319e-18j],
       [0.1186949 -1.17808971e-18j, 0.31099718-8.84392895e-02j,
        0.31099718+8.84392895e-02j, 0.88130531+1.26516016e-18j],
       [0.11918453+1.91714554e-18j, 0.31162905-8.83246533e-02j,
        0.31162905+8.83246533e-02j, 0.88081568-7.48351412e-18j]])

In [11]:
newX = np.zeros(X.shape)
for i in range(X.shape[0]):
    newX[i] = np.array([X[i][0].real, X[i][1].real, X[i][1].imag, X[i][3].real])

In [13]:
newy = np.zeros((y.shape[0],4))
for i in range(y.shape[0]):
    newy[i] = np.array([y[i][0].real, y[i][0].imag, y[i][1].real, y[i][1].imag])

In [14]:
X = newX
y = newy

In [15]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.10, random_state=57109, shuffle=True)

In [16]:
train_datat = torch.from_numpy(train_data)
train_labelst = torch.from_numpy(train_labels)
test_datat = torch.from_numpy(test_data)
test_labelst = torch.from_numpy(test_labels)

In [17]:
hidden_size = 100
net = Net(input_size, hidden_size, output_size)
net = net.to(device)

In [18]:
train_datat = train_datat.to(device)
train_labelst = train_labelst.to(device)
test_datat = test_datat.to(device)
test_labelst = test_labelst.to(device)

In [30]:
from torch.nn.functional import mse_loss
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [31]:
num_epochs = 1000
batch_size = 256
losses = []
train_acc = []
for epoch in range(1, num_epochs + 1):
        final_loss = 0
        for i in range(0, train_datat.shape[0], batch_size):
            optimizer.zero_grad()
            if i + batch_size > train_datat.shape[0]:
                batched_data = train_datat[i:]
            else:
                batched_data = train_datat[i: i + batch_size]
            outputs = net(batched_data)
            loss =  mse_loss(outputs, train_labelst[i: i + batch_size])
            final_loss = loss
            loss.backward()
            optimizer.step()
        losses.append(loss)
        train_acc.append(train_score) 
        if epoch % 100 == 0:
            print('Epoch: {}/{}.............'.format(epoch, num_epochs), end=' ')
            print("Loss: {:.4f}".format(loss.item()))

/tmp/ipykernel_18334/3876127444.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sm(out)


Epoch: 100/1000............. Loss: 0.0031
Epoch: 200/1000............. Loss: 0.0031
Epoch: 300/1000............. Loss: 0.0031
Epoch: 400/1000............. Loss: 0.0031
Epoch: 500/1000............. Loss: 0.0031
Epoch: 600/1000............. Loss: 0.0031
Epoch: 700/1000............. Loss: 0.0031
Epoch: 800/1000............. Loss: 0.0031
Epoch: 900/1000............. Loss: 0.0031
Epoch: 1000/1000............. Loss: 0.0031


In [35]:
net(torch.from_numpy(newX).to(device))

/tmp/ipykernel_18334/3876127444.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sm(out)


tensor([[9.9796e-01, 5.3200e-39, 1.9804e-03, 6.1775e-05],
        [9.9969e-01, 2.4524e-37, 2.0859e-04, 1.0014e-04],
        [9.9926e-01, 2.3882e-31, 2.8194e-04, 4.5955e-04],
        ...,
        [9.9948e-01, 1.6088e-37, 5.1086e-04, 1.2386e-05],
        [9.9891e-01, 2.2842e-37, 1.0915e-03, 2.8661e-06],
        [9.9921e-01, 5.5645e-36, 7.4707e-04, 4.0469e-05]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [36]:
newy

array([[ 1.0000000e+00,  3.4200000e-06, -1.0000000e-08, -2.7000000e-06],
       [ 1.0000000e+00,  1.0270000e-05, -6.0000000e-08, -8.1000000e-06],
       [ 1.0000000e+00,  2.0530000e-05, -1.7000000e-07, -1.6200000e-05],
       ...,
       [ 9.9182448e-01,  8.7885200e-03, -1.7524880e-02, -1.2609455e-01],
       [ 9.9182448e-01,  8.7885200e-03, -1.8664610e-02, -1.2593089e-01],
       [ 9.9182448e-01,  8.7885200e-03, -1.9802810e-02, -1.2575693e-01]])

In [43]:
x = net(torch.from_numpy(newX).to(device)).cpu()
y = newy
np.linalg.norm((x.detach().numpy()-y))

/tmp/ipykernel_18334/3876127444.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.sm(out)


10.93615856811218